In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Copy of data.csv')
df.head()

,Unnamed: 0,Index,File name,Context,Judgement
0,0,0,J1.pdf,reportable in the supreme court of india civil...,Ambiguity
1,1,1,J10.pdf,"In the present facts, it is clear that BCCL an...",Respondent
2,2,2,J11.PDF,It is true that the ARC is not a party-respond...,Ambiguity
3,3,3,J12.PDF,"Hence, it has to be directed that till such am...",Petitioner
4,4,4,J13.PDF,"In view of above, we are inclined to allow thi...",Petitioner


In [ ]:
import random
random.seed(50)

In [ ]:
#1
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/electra-small-discriminator")

In [ ]:
def process_data(row):

    text = row['Context']
    text = str(text)
    text = ' '.join(text.split())

    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=512)

    label = 0
    if row['Judgement'] == 'Petitioner':
        label += 1
    if row['Judgement'] == 'Respondent':
        label += 2
    encodings['label'] = label
    encodings['text'] = text

    return encodings

In [ ]:
print(process_data({
    'Context': 'this is a sample context of the document.',
    'Judgement': 'Petitioner'
}))

{'input_ids': [101, 2023, 2003, 1037, 7099, 6123, 1997, 1996, 6254, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
processed_data = []

for i in range(len(df[:1000])):
    processed_data.append(process_data(df.iloc[i]))

In [ ]:
processed_data

In [ ]:
#split the data into train and validation data 
from sklearn.model_selection import train_test_split

new_df = pd.DataFrame(processed_data)

train_df, valid_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=2022
)

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyarrow as pa
from datasets import Dataset

train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

## using gelu activation function

In [ ]:
#1
from transformers import AutoModelForSequenceClassification
model1 = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator",num_labels=3,hidden_act="gelu",attention_probs_dropout_prob =0.3,hidden_dropout_prob=0.3)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

In [ ]:
type(model)

transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification

In [ ]:
from datasets import load_metric
m1 = load_metric('accuracy')
acc = []


def compute_metrics(eval_pred,acc = acc):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    x = m1.compute(predictions=predictions, references=labels)
    acc.append(x)
    return x

<ipython-input-25-285e81eca82c>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  m1 = load_metric('accuracy')


In [ ]:
from transformers import TrainingArguments, Trainer
from transformers.optimization import Adafactor, AdafactorSchedule

optimizer = Adafactor(model1.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)
lr_scheduler = AdafactorSchedule(optimizer)
training_args = TrainingArguments(output_dir=f"./out_fold{i}",
                                  overwrite_output_dir = 'True', 
                                  evaluation_strategy="epoch",
                                  eval_steps=10,
                                  logging_steps = 10,
                                  num_train_epochs=100,
                                  logging_dir="./logs",
                                  load_best_model_at_end = True,
                                  save_strategy= 'epoch'
      )

trainer = Trainer(
    model=model1,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer,
    optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()
trainer.evaluate()

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.103100,1.107269,0.075000
2,1.102800,1.103119,0.125000
3,1.096700,1.097094,0.375000
4,1.085500,1.088224,0.475000
5,1.068700,1.075174,0.425000
6,1.053200,1.065607,0.450000
7,1.068700,1.056180,0.425000
8,1.049600,1.041447,0.425000
9,1.047900,1.030041,0.425000
10,1.027300,1.019066,0.425000


{'eval_loss': 0.8084114193916321,
 'eval_accuracy': 0.725,
 'eval_runtime': 0.2839,
 'eval_samples_per_second': 140.897,
 'eval_steps_per_second': 17.612,
 'epoch': 100.0}

In [ ]:
model1.save_pretrained('./out_fold1990/checkpoint-100/')
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('./out_fold1990/checkpoint-100/')

In [ ]:
model1.save_pretrained('./model/')
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('./model/')

## using RELU activation function


In [ ]:
#2
from transformers import AutoModelForSequenceClassification
model1 = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator",num_labels=3,hidden_act="relu",attention_probs_dropout_prob =0.3,hidden_dropout_prob=0.3)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers.optimization import Adafactor, AdafactorSchedule

optimizer = Adafactor(model1.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)
lr_scheduler = AdafactorSchedule(optimizer)
training_args = TrainingArguments(output_dir=f"./out_fold{i}",
                                  overwrite_output_dir = 'True', 
                                  evaluation_strategy="epoch",
                                  eval_steps=10,
                                  logging_steps = 10,
                                  num_train_epochs=100,
                                  logging_dir="./logs",
                                  load_best_model_at_end = True,
                                  save_strategy= 'epoch'
      )

trainer = Trainer(
    model=model1,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer,
    optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()
trainer.evaluate()
model1.save_pretrained('./model/')
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('./model/')

Epoch,Training Loss,Validation Loss,Accuracy
1,1.114100,1.117655,0.050000
2,1.113900,1.114603,0.050000
3,1.098800,1.109547,0.050000
4,1.097200,1.103343,0.425000
5,1.099100,1.095958,0.425000
6,1.094800,1.087967,0.425000
7,1.083500,1.079514,0.425000
8,1.077500,1.073419,0.425000
9,1.070400,1.060242,0.425000
10,1.061200,1.047796,0.425000


## using silu activation function


In [ ]:
#3
from transformers import AutoModelForSequenceClassification
model1 = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator",num_labels=3,hidden_act="silu",attention_probs_dropout_prob =0.1,hidden_dropout_prob=0.1)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers.optimization import Adafactor, AdafactorSchedule

optimizer = Adafactor(model1.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)
lr_scheduler = AdafactorSchedule(optimizer)
training_args = TrainingArguments(output_dir=f"./out_fold{i}",
                                  overwrite_output_dir = 'True', 
                                  evaluation_strategy="epoch",
                                  eval_steps=10,
                                  logging_steps = 10,
                                  num_train_epochs=100,
                                  logging_dir="./logs",
                                  load_best_model_at_end = True,
                                  save_strategy= 'epoch')

trainer = Trainer(
    model=model1,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer,
    optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()
trainer.evaluate()
model1.save_pretrained('./model/')
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('./model/')

Epoch,Training Loss,Validation Loss,Accuracy
1,1.094100,1.083282,0.525000
2,1.074200,1.067524,0.425000
3,1.068600,1.053678,0.425000
4,1.058100,1.033490,0.425000
5,1.035000,1.014224,0.425000
6,1.004000,0.997224,0.425000
7,1.023300,0.978011,0.575000
8,1.007700,0.969129,0.450000
9,0.993000,0.943614,0.575000
10,0.987700,0.935285,0.550000


In [ ]:
trainer.evaluate()

{'eval_loss': 0.5794187188148499,
 'eval_accuracy': 0.8,
 'eval_runtime': 0.2846,
 'eval_samples_per_second': 140.544,
 'eval_steps_per_second': 17.568,
 'epoch': 100.0}

## using gelu new activation function


In [ ]:
from transformers import AutoModelForSequenceClassification
model1 = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator",num_labels=3,hidden_act="gelu_new",attention_probs_dropout_prob =0.3,hidden_dropout_prob=0.3)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers.optimization import Adafactor, AdafactorSchedule

optimizer = Adafactor(model1.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)
lr_scheduler = AdafactorSchedule(optimizer)
training_args = TrainingArguments(output_dir=f"./out_fold{i}",
                                  overwrite_output_dir = 'True', 
                                  evaluation_strategy="epoch",
                                  eval_steps=10,
                                  logging_steps = 10,
                                  num_train_epochs=100,
                                  logging_dir="./logs",
                                  load_best_model_at_end = True,
                                  save_strategy= 'epoch')

trainer = Trainer(
    model=model1,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer,
    optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()
trainer.evaluate()
model1.save_pretrained('./model/')
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('./model/')

Epoch,Training Loss,Validation Loss,Accuracy
1,1.099200,1.094331,0.350000
2,1.088300,1.090997,0.350000
3,1.087700,1.085564,0.375000
4,1.086100,1.078272,0.325000
5,1.076700,1.070281,0.300000
6,1.060900,1.061034,0.425000
7,1.063000,1.049806,0.450000
8,1.050000,1.036782,0.425000
9,1.038500,1.022733,0.450000
10,1.035800,1.013594,0.425000


In [ ]:
trainer.evaluate()

{'eval_loss': 0.5808665752410889,
 'eval_accuracy': 0.85,
 'eval_runtime': 0.4144,
 'eval_samples_per_second': 96.532,
 'eval_steps_per_second': 12.067,
 'epoch': 100.0}

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./model/') 

In [ ]:
from transformers import AutoTokenizer

new_tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [ ]:
import torch
import numpy as np
if torch.cuda.is_available():  # Tell PyTorch to use the GPU. 
 device = torch.device("cuda") 
 print('There are %d GPU(s) available.' % torch.cuda.device_count()) 
 print('We will use the GPU:', torch.cuda.get_device_name(0)) # If not...
else:
 print('No GPU available, using the CPU instead.')
 device = torch.device("cpu")
model.to(device)
def get(text):
    encoding = new_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    encoding = {k:v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = trainer.model( **encoding)

    logits = outputs.logits

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    probs = probs.detach().numpy()
    label = np.argmax(probs, axis=-1)
    
    if label == 1:
        return {
            'w': 'Petitioner',
            'probability': probs[1]
        }
    elif label == 2:
        return {
            'w': 'respondent',
            'probability': probs[2]
            }
    elif label == 0:
        return {
            'w': 'am',
            'probability': probs[0]
            }

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
a = get('We therefore allow this appeal, set aside the impugned order of the High Court and quash the criminal proceedings of Criminal Complaint No. 1029 of 2015. We clarify that this will not come in the way of instituting appropriate proceedings in future in case the Civil Court comes to the conclusion that the disputed sale deed dated 29.12.2010 is forged. We refrain ourselves from expressing any opinion as regards the genuineness or otherwise of the sale deed in question as this question is wide open before the Civil Court. The Civil Court shall decide the civil suit pending between the parties on its own merits and on the basis of the evidence that may be led by both the sides. It shall be open to the Civil Court to take the opinion of the hand writing expert as regards the signature of the complainant on the disputed sale deed. We clarify that we have passed the aforesaid order in the facts and circumstances of the present case and the same shall not be cited as a precedent.')
a['w']

'Petitioner'

In [ ]:
b = get('For the above reasons, we hold that no prejudice has resulted to the respondent on account of not furnishing him the copies of the statements of witnesses. We are satisfied that on account of the said violations it cannot he said that the respondent did not have a fair hearing or that the disciplinary enquiry against him was not a fair enquiry. Accordingly, we allow the appeal and set aside the judgment of the High Court affirming the judgments of the Trial Court and Appellate Court.')
b['w']

'Petitioner'